This Jupyter notebook is used to check that the DistortedConv2D behaves as expected. To do this, we will pass in an input image and convolve this with a simple sobel edge detector.

In [3]:
from distorted_convolution import DistortedConv2D
from distorter import Distorter
import numpy as np

Reading the input panorama file.

In [51]:
# Loads the input image.
from matplotlib.image import imread
from matplotlib import pyplot as plt

img_height = 346
img_width = 1024
batch_size = 1
train_ds = tf.keras.utils.image_dataset_from_directory(
  "data",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
print(data.shape)


Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: No images found in directory data. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [46]:
import numpy as np
import torch
from torch import nn
from torch.nn.parameter import Parameter
from distorter import Distorter
import tensorflow as tf


class DistortedConv2D(nn.Module):
    ''' Performs convolution with distorted sampling locations. '''
    def __init__(self, in_c, out_c, stride=1, mode="bilinear", kernel_size=3, bias = True):
        ''' Initialises the weights '''
        super(DistortedConv2D, self).__init__()
        self.in_c = in_c 
        self.out_c = out_c
        self.stride = stride
        self.mode = mode
        self.kernel_size = kernel_size

        # Initialises the weights
        self.weights = Parameter(torch.Tensor(out_c, in_c, kernel_size, kernel_size))
        if bias:
            self.bias = Parameter(torch.Tensor(out_c))
        else:
            self.register_parameter('bias', None)
        self.grid_shape = None
        self.grid = None

        self.clear_weights_biases()

    def clear_weights_biases(self):
        ''' Resets the weights to their default'''
        nn.init.xavier_uniform(self.weights)
        self.bias.data.zero_()

    def forward(self, input):
        # Dimensions of the image.
        IMAGE_HEIGHT = input.shape[1]
        IMAGE_WIDTH = input.shape[2]
        
        # If the grid shape doesn't match the shape of the image.
        if self.grid_shape is None or self.grid_shape != tuple(input.shape[1:3]):
            # Distorts the coordinates.
            distorter = Distorter(IMAGE_WIDTH, IMAGE_HEIGHT, self.kernel_size)
            print("Input shape", input.shape[2:4])
            self.grid_shape = tuple(input.shape[2:4])
            coordinates = distorter.distort_all_points()
            with torch.no_grad():
                self.grid = torch.FloatTensor(coordinates).to(device="cpu")
                self.grid.requires_grad = True
#                 print("Grid shape ", self.grid.shape)
#                 print("Distorted grid = ", self.grid)

        with torch.no_grad():
            self.grid = self.grid.repeat(input.shape[0], 1, 1, 1)
        print(type(self.grid))
        sampled = nn.functional.grid_sample(input, self.grid, mode=self.mode)
        output = nn.functional.conv2d(sampled, self.weight, self.bias, stride=self.kernel_size)
        return output

    #TODO test the convolution

In [47]:
conv1 = DistortedConv2D(64, 32, stride=1)
conv1(data)

<ipython-input-46-483dd65a62fd>:33: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.weights)


Input shape (346, 1024)
(346, 1024, 3, 3, 2)
<class 'torch.Tensor'>


TypeError: grid_sampler(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
""" This file is used to test the implemented distorted awareness convolution operation. """

# TODO implement an edge detector template

# TODO Create a distortion awareness convolution filter and pass the image in.

# TODO Create a traditional convolution filter and pass the 360 image in.

# TODO Save the output images as files and compare them manually.

# TODO Repeat the above but with SphereNet.